In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
# !pip3 install tensorflow-gpu==1.14

In [3]:
import tensorflow as tf
import numpy as np
from tensor2tensor.data_generators import problem_hparams
from tensor2tensor.models import evolved_transformer
from tensor2tensor.models import transformer

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [4]:
import json

with open('train-test-bpe.json') as fopen:
    dataset = json.load(fopen)

In [5]:
train_X = dataset['train_X']
train_Y = dataset['train_Y']
test_X = dataset['test_X']
test_Y = dataset['test_Y']

In [6]:
params = transformer.transformer_tiny()
english_size = 32000
vietnam_size = 32000
p_hparams = problem_hparams.test_problem_hparams(english_size, vietnam_size, params)
params.problem_hparams = p_hparams

In [7]:
tf.estimator.ModeKeys.PREDICT

'infer'

In [8]:
def pad_second_dim(x, desired_size):
    padding = tf.tile([[[0.0]]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1], tf.shape(x)[2]], 0))
    return tf.concat([x, padding], 1)

class Translator:
    def __init__(self, learning_rate = 1e-3, beam_size = 5, training = True):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        self.training = tf.placeholder(tf.bool, None)
        batch_size = tf.shape(self.X)[0]
        seq_size = tf.shape(self.X)[1]
        
        maxlen_decode = 2 * tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        print(x, y)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        
        if training:
            mode = 'train'
        else:
            mode = 'infer'
        
        model = evolved_transformer.EvolvedTransformer(params,mode,p_hparams)
        
        logits, _ = model(features)
        logits = tf.squeeze(tf.squeeze(logits, 2), 2)
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        
        y_t = tf.argmax(logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
        
            self.fast_result = model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = model._beam_decode_slow(
                features, maxlen_decode, beam_size=beam_size, top_beams=1, alpha=1.0)["outputs"]
            
            print(self.fast_result, self.beam_result)

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator()
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


Tensor("ExpandDims_1:0", shape=(?, ?, 1, 1), dtype=int32) Tensor("ExpandDims_3:0", shape=(?, ?, 1, 1), dtype=int32)



INFO:tensorflow:Setting T2TModel mode to 'train'


INFO:tensorflow:Setting T2TModel mode to 'train'


...

INFO:tensorflow:Building model body


INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_32000_128.top


INFO:tensorflow:Transforming body output with symbol_modality_32000_128.top


Tensor("evolved_transformer_1/while/Exit_3:0", shape=(?, ?), dtype=int64) Tensor("strided_slice_12:0", shape=(?, ?), dtype=int32)


In [10]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [11]:
batch_x, _ = pad_sentence_batch(train_X[: 5], 0)
batch_y, _ = pad_sentence_batch(train_Y[: 5], 0)

sess.run([model.cost, model.optimizer], feed_dict = {model.X: batch_x, model.Y: batch_y})

[10.888551, None]

In [12]:
batch_size = 96
epoch = 20
PAD = 0

In [13]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x, seq_x = pad_sentence_batch(train_X[i : index], PAD)
        batch_y, seq_y = pad_sentence_batch(train_Y[i : index], PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
        
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x, seq_x = pad_sentence_batch(test_X[i : index], PAD)
        batch_y, seq_y = pad_sentence_batch(test_Y[i : index], PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 1/1389 [00:00<03:27,  6.68it/s, accuracy=0.347, cost=3.96]

epoch 1, training avg loss 4.941576, training avg acc 0.238279
epoch 1, testing avg loss 4.067009, testing avg acc 0.331084


minibatch loop:   0%|          | 1/1389 [00:00<03:26,  6.73it/s, accuracy=0.438, cost=3.22]

epoch 2, training avg loss 3.502500, training avg acc 0.398134
epoch 2, testing avg loss 3.405555, testing avg acc 0.413942


minibatch loop:   0%|          | 1/1389 [00:00<03:28,  6.65it/s, accuracy=0.483, cost=2.87]

...

epoch 18, training avg loss 1.980102, training avg acc 0.578285
epoch 18, testing avg loss 2.787682, testing avg acc 0.497013


minibatch loop:   0%|          | 1/1389 [00:00<03:22,  6.87it/s, accuracy=0.582, cost=2]

epoch 19, training avg loss 1.964271, training avg acc 0.580201
epoch 19, testing avg loss 2.790944, testing avg acc 0.495617


minibatch loop: 100%|██████████| 30/30 [00:02<00:00, 11.33it/s, accuracy=0.436, cost=3.23]

epoch 20, training avg loss 1.950056, training avg acc 0.582312
epoch 20, testing avg loss 2.785332, testing avg acc 0.497402


In [15]:
import youtokentome as yttm
bpe = yttm.BPE(model='vietnam.model')

In [37]:
greedy = sess.run(model.fast_result, feed_dict = {model.X: test_X[:1]})

In [38]:
actual = test_Y[0]
predicted = list(dict.fromkeys(greedy[0].tolist()))

actual, predicted

([1377,
  577,
  278,
  281,
  323,
  627,
  1479,
  321,
  316,
  1196,
  353,
  2097,
  1485,
  818,
  631,
  288,
  317,
  784,
  807,
  575,
  942,
  365,
  620,
  256,
  353,
  403,
  346,
  365,
  440,
  288,
  2097,
  1485,
  896,
  1687,
  873,
  656,
  1230,
  545,
  327,
  12091,
  2041,
  859,
  527,
  513,
  291,
  279,
  670,
  858,
  1517,
  827,
  2944,
  299,
  794,
  334,
  975,
  800,
  1195,
  723,
  291,
  670,
  858,
  256,
  334,
  986,
  270,
  969,
  321,
  1208,
  449,
  582,
  1522,
  437,
  429,
  2207,
  723,
  11296,
  19468,
  437,
  5521,
  1762,
  415,
  316,
  449,
  596,
  256,
  288,
  2097,
  1485,
  818,
  631,
  327,
  12091,
  859,
  527,
  888,
  670,
  858,
  545,
  270,
  302,
  710,
  512,
  1701,
  453,
  333,
  333,
  670,
  858,
  545,
  512,
  513,
  348,
  2385,
  3354,
  270,
  401,
  381,
  576,
  334,
  297,
  318,
  1056,
  699,
  392],
 [1377,
  365,
  440,
  351,
  381,
  321,
  316,
  1196,
  353,
  288,
  317,
  784,
  807,
  620,

In [39]:
bpe.decode(actual)

['Làm sao tôi có thể trình bày trong <NUM> phút về sợi dây liên kết những người phụ nữ qua ba thế hệ , về việc làm thế nào những sợi dây mạnh mẽ đáng kinh ngạc ấy đã níu chặt lấy cuộc sống của một cô bé bốn tuổi co quắp với đứa em gái nhỏ của cô bé , với mẹ và bà trong suốt năm ngày đêm trên con thuyền nhỏ lênh đênh trên Biển Đông hơn <NUM> năm trước , những sợi dây liên kết đã níu lấy cuộc đời cô bé ấy và không bao giờ rời đi - - cô bé ấy giờ sống ở San Francisco và đang nói chuyện với các bạn hôm nay ?']

In [40]:
bpe.decode(predicted)

['Làm thế nào để nói trong <NUM> phút về những người phụ nữ hệ , việc có sức mạnh đáng kinh ngạc đời sống tuổi của cô mẹ bé ngày một chiếc thuyền nhỏ ở Trung Quốc hơn năm trước liên kết giữ cho con gái và không bao giờ đi . " đang San Francisco chuyện với ấy nay ? bây là bà già tháng lại giao này Cô nam nữa hôm - đàn ông đã nắm cuộc làm']